In [4]:
import json

# Load questions from JSON file
with open('data/eval_data/questions.json', 'r') as file:
    questions_data = json.load(file)

questions_en = questions_data['en']
questions_de = questions_data['de']


In [5]:
from openai import OpenAI
import pickle
from dotenv import load_dotenv

load_dotenv()

# Initialize the OpenAI API client
client = OpenAI()

# Load the vector store ID and file paths from the pickle file
with open('vector_store.pkl', 'rb') as f:
    data = pickle.load(f)
vector_store_id = data['vector_store_id']

# Create and update the assistant
assistant = client.beta.assistants.create(
    name="Product Documentation Assistant",
    instructions="""You are a knowledgeable product support assistant for specific lighting products. 
    Use the provided documents to answer user queries about product specifications, usage guidelines, 
    and other relevant details ONLY for the products mentioned in these documents.
    Make sure you answet in the language of the provided query. 
    If a question is about any topic or product not covered in the provided documents, 
    including other companies products, respond with "I don't have information about that." 
    I can only provide details about the specific lighting products in my documentation.'""",
    model="gpt-4o",    
    tools=[{"type": "file_search"}],
)


assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}},
)

# Function to query questions and save responses
def query_questions(questions, language):
    results = []
    for question in questions:
        thread = client.beta.threads.create(
            messages=[
                {
                    "role": "user",
                    "content": question,
                }
            ]
        )

        # Create and poll the run
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, assistant_id=assistant.id
        )

        # List messages and capture response
        messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
        message_content = messages[0].content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        answer = {
            "question": question,
            "answer": message_content.value,
            "citations": citations,
            "language": language
        }
        results.append(answer)
    return results




In [6]:
# Query the questions in English and German
results_en = query_questions(questions_en, "en")
results_de = query_questions(questions_de, "de")

# Save the results to a new JSON file
with open('questions_answers_de.json', 'w') as file:
    json.dump(results_de, file, ensure_ascii=False, indent=4)

with open('questions_answers_en.json', 'w') as file:
    json.dump(results_en, file, ensure_ascii=False, indent=4)

# Combine results
all_results = results_en + results_de
with open('questions_answers.json', 'w') as file:
    json.dump(all_results, file, ensure_ascii=False, indent=4)


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


client = OpenAI()

# Load the saved questions and answers from the JSON file
with open('data/eval_data/questions_answers.json', 'r') as file:
    qa_data = json.load(file)

# Function to get embedding from OpenAI
def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

# Function to evaluate the responses
def evaluate_responses(qa_data):
    evaluation_results = []
    for item in qa_data:
        question = item['question']
        answer = item['answer']

        # Compute embeddings for question and answer
        question_embedding = get_embedding(question)
        answer_embedding = get_embedding(answer)

        # Compute cosine similarity
        similarity_score = cosine_similarity(
            [question_embedding], [answer_embedding]
        )[0][0]

        evaluation_results.append({
            "question": question,
            "answer": answer,
            "similarity_score": similarity_score,
            "citations": item.get('citations', []),
            "language": item['language']
        })

    return evaluation_results

# Evaluate the responses
evaluation_results = evaluate_responses(qa_data)

# Save the evaluation results to a new JSON file
with open('evaluation_results.json', 'w') as file:
    json.dump(evaluation_results, file, ensure_ascii=False, indent=4)

# Print some sample evaluation results
for result in evaluation_results[:5]:  
    print(f"Question: {result['question']}")
    print(f"Answer: {result['answer']}")
    print(f"Similarity Score: {result['similarity_score']}")
    print(f"Citations: {result['citations']}")
    print(f"Language: {result['language']}")
    print()

# Compute and print average similarity scores for each language
similarity_scores_en = [result['similarity_score'] for result in evaluation_results if result['language'] == 'en']
similarity_scores_de = [result['similarity_score'] for result in evaluation_results if result['language'] == 'de']

average_similarity_en = np.mean(similarity_scores_en)
average_similarity_de = np.mean(similarity_scores_de)

print(f"Average Similarity Score for English Questions: {average_similarity_en}")
print(f"Average Similarity Score for German Questions: {average_similarity_de}")


Question: What is the nominal current of the XBO 2500 W/HS XL OFR lamp?
Answer: The nominal current of the XBO 2500 W/HS XL OFR lamp is 90.00 A[0].
Similarity Score: 0.9489970732787987
Citations: ['[0] ZMP_1007189_XBO_2500_W_HS_XL_OFR.pdf']
Language: en

Question: How long is the XBO 2500 W/HS XL OFR lamp?
Answer: The length of the XBO 2500 W/HS XL OFR lamp is 342.0 mm[0].
Similarity Score: 0.8903802089344491
Citations: ['[0] ZMP_1007189_XBO_2500_W_HS_XL_OFR.pdf']
Language: en

Question: What are the electrical specifications of the XBO 3000 W/H XL OFR lamp?
Answer: The electrical specifications of the XBO 3000 W/H XL OFR lamp are as follows:

- **Nominal Current:** 105 A
- **Current Control Range:** 70…110 A
- **Nominal Power:** 3000 W
- **Nominal Voltage:** 28.0 V[0].
Similarity Score: 0.9264885073200801
Citations: ['[0] ZMP_1007191_XBO_3000_W_H_XL_OFR.pdf']
Language: en

Question: What is the lifespan of the XBO 3000 W/H XL OFR lamp?
Answer: The lifespan of the XBO 3000 W/H XL OFR l